Required libraries is cv2 and deepface. You can install them by running the following command:
```bash
pip install opencv-python
pip install deepface
```
and download the pre-trained face detection model YuNet from the following link:



In [7]:
import cv2
from deepface import DeepFace
import os

In [15]:

# Add new face to the database
def add_to_database(name, db_path, frame):
    img_name = f"{name}.jpeg"  # Avoid special characters like "_"
    img_path = os.path.join(db_path, img_name)  # Create path for the image
    cv2.imwrite(img_path, frame)  # Capture image and add to the database
    print(f"{name} added to the database.")

# Delete faces from the database
def delete_faces(name, db_path):
    img_name = f"{name}.jpeg"  # Same naming convention
    img_path = os.path.join(db_path, img_name)
    if os.path.isfile(img_path):
        os.remove(img_path)
        print(f"{name}'s image deleted from the database.")
    else:
        print(f"{name} does not exist in the database.")
        
#Change the path to the database folder
db_path = r"E:\CSE\Mind Cloud\Mind Github\Final-Project-Mini-Smart-Home-System\OpenCV&FaceRecognition\Images"  # Use raw string for Windows paths        

Capture personal photo and add it to the Images folder (Database)

In [33]:
ret, frame = cv2.VideoCapture(0).read()  # Capture a frame
add_to_database("ebrahim", db_path, frame)

ebrahim added to the database.


In [34]:

# Initialize video capture from the webcam
cap = cv2.VideoCapture(0)
        
#Change the path to the database folder
db_path = r"E:\CSE\Mind Cloud\Mind Github\Final-Project-Mini-Smart-Home-System\OpenCV&FaceRecognition\Images"  # Use raw string for Windows paths        
# # Path to the face recognition database #### Change the path to the location of your database
# db_path = r"E:\CSE\Mind Cloud\Mind Github\Final-Project-Mini-Smart-Home-System\OpenCV & FaceRecognition\Images"

# Initialize the face detector (FaceDetectorYN) with the ONNX model
detector = cv2.FaceDetectorYN.create(
    model='face_detection_yunet_2023mar.onnx',  # Path to the ONNX model
    config='',  # Additional configurations (optional)
    input_size=(640, 480),  # Input size for the model
    score_threshold=0.5,    # Confidence threshold for detections
    nms_threshold=0.4       # Non-Maximum Suppression threshold
)

# Constants and variables
face_detected = False  # Flag to track if a face was detected
name = "Unknown"  # Default name for detected faces
temp_image_path = 'detected_face.jpg'  # Temp file to store the detected face

while True:
    ret, frame = cap.read()  # Capture frame-by-frame
    if not ret:
        print("Failed to capture frame.")
        break

    # Detect faces in the frame
    _, faces = detector.detect(frame)

    if faces is not None and len(faces) > 0:
        for face in faces:
            # Extract bounding box coordinates
            x, y, w, h = face[0:4].astype(int)

            # Ensure the coordinates are valid and within the frame bounds
            x, y, w, h = max(0, x), max(0, y), max(1, w), max(1, h)

            # Extract face region (ROI) from the frame
            out_image = frame[y:y+h, x:x+w]

            # Save the face region as a temporary image
            cv2.imwrite(temp_image_path, out_image)

            # Use DeepFace to perform face recognition
            try:
                res = DeepFace.find(img_path=temp_image_path, db_path=db_path, enforce_detection=False, model_name="Facenet")

                if len(res[0]) > 0:
                    # Face found, extract name and bounding box
                    name = os.path.basename(res[0]['identity'][0]).split("_")[0]
                    bounding_box = (x, y, x + w, y + h)
                    face_detected = True
                    print(f"Found {name}, opening the door.")
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                else:
                    name = "Unknown"

            except Exception as e:
                # Handle cases where DeepFace fails to recognize the face
                print(f"Error during face recognition: {e}")
                name = "Unknown"
                bounding_box = None

            # If no face is recognized, display "Unknown"
            if name == "Unknown":
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(frame, "Unknown", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    

    # Display the resulting frame with annotations
    cv2.imshow("Result", frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


24-09-09 03:44:29 - Found 0 newly added image(s), 0 removed image(s), 2 replaced image(s).


Finding representations: 100%|██████████| 2/2 [00:00<00:00,  3.93it/s]


24-09-09 03:44:29 - There are now 3 representations in ds_model_facenet_detector_opencv_aligned_normalization_base_expand_0.pkl
24-09-09 03:44:29 - Searching detected_face.jpg in 3 length datastore
24-09-09 03:44:30 - find function duration 0.7569983005523682 seconds
Found ebrahim.jpeg, opening the door.
24-09-09 03:44:30 - Searching detected_face.jpg in 3 length datastore
24-09-09 03:44:30 - find function duration 0.24000072479248047 seconds
Found ebrahim.jpeg, opening the door.
24-09-09 03:44:30 - Searching detected_face.jpg in 3 length datastore
24-09-09 03:44:30 - find function duration 0.22500300407409668 seconds
24-09-09 03:44:30 - Searching detected_face.jpg in 3 length datastore
24-09-09 03:44:31 - find function duration 0.22499990463256836 seconds
24-09-09 03:44:31 - Searching detected_face.jpg in 3 length datastore
24-09-09 03:44:31 - find function duration 0.22500276565551758 seconds
24-09-09 03:44:31 - Searching detected_face.jpg in 3 length datastore
24-09-09 03:44:31 - fi